## Extracting Location Data for Each NFL team

In [1]:
#read in list of NFL teams
with open('teams.txt', encoding='utf-8-sig') as team_file:
    teams = [line.strip() for line in team_file]
teams

#read in the list of split tweet files.
#I broke these up because I don't think my computer would know what to do when trying to load 18GB into RAM.
#Using utf-8-sig encoding because when I saved as a UTF-8 encoded file, it saved it as UTF-8-BOM for some reason.
with open("file_names.txt", encoding = "utf-8-sig") as file_names_file:
        file_names = [line.strip() for line in file_names_file]

In order to get this into a nice format for R, we'll just create a list that duplicates team names for applicable coordiates. I know that a dictionary of lists is much more efficient, but that makes it more difficult to port over to R.

In [2]:
team_coords = []

The following 4 code chunks test to make sure I'm getting the coordinates correctly.

In [3]:
with open('test_coords.txt', encoding='utf-8-sig') as coords_file:
    tweet = coords_file.readline()

In [4]:
#Time to work with regular expressions
import re

In [5]:
def get_coords(tweet):
    try:
        found = re.search('coordinates\': \[\[\[(.*?)\]', tweet).group(1)
    except AttributeError:
        # if no coordinates found
        found = '' # return blank string
    return(found)

In [6]:
get_coords(tweet)

'-73.701496, 40.582128'

In [7]:
#This chews through each of the 1/4 GB files until all 18 GBs are processed.
for file in file_names:
    with open(file, encoding = "utf-8-sig") as curr_tweets:
        tweets = curr_tweets.readlines()
        for i in range(len(tweets)):
            coords = get_coords(tweets[i])
            if coords != '':
                for curr_team in teams:
                    if curr_team in tweets[i]:
                        team_coords.append((curr_team, coords))

In [8]:
team_coords

[('Lions', '-112.014232, 33.508746'),
 ('Packers', '-112.014232, 33.508746'),
 ('Lions', '-112.014232, 33.508746'),
 ('Bears', '-81.046876, 35.001706'),
 ('Vikings', '-93.329515, 44.889964'),
 ('Jets', '-96.85743, 32.997151'),
 ('Jets', '-74.041878, 40.570842'),
 ('Vikings', '-93.329515, 44.889964'),
 ('Browns', '-73.962582, 40.541722'),
 ('Titans', '-73.962582, 40.541722'),
 ('Cowboys', '-93.290637, 45.035336'),
 ('Bills', '-97.239256, 43.499362'),
 ('Steelers', '-85.605166, 30.355644'),
 ('Saints', '-82.588866, 27.293114'),
 ('Broncos', '-114.316038, 50.84278'),
 ('Patriots', '-71.177553, 42.135115'),
 ('49ers', '-88.139804, 44.47174'),
 ('Panthers', '-75.280284, 39.871811'),
 ('Cowboys', '-95.846367, 29.676113'),
 ('Giants', '-95.846367, 29.676113'),
 ('Ravens', '-76.976222, 39.448205'),
 ('Bills', '-76.976222, 39.448205'),
 ('Packers', '-79.487651, 37.886607'),
 ('Redskins', '-79.487651, 37.886607'),
 ('Bears', '-89.671507, 42.581987'),
 ('Bills', '-97.239256, 43.499362'),
 ('Texan

In [9]:
import pandas as pd

In [10]:
coords_df = pd.DataFrame(team_coords, columns=['Teams', 'Coordinates'])

Split coordinates into longtitude and latitude columns.

In [11]:
long_lat = coords_df['Coordinates'].str.split(',', expand=True)
coords_df = pd.concat([coords_df, long_lat], axis=1)
coords_df.columns = ['Teams', 'Coordinates', 'Longitude', 'Latitude']
coords_df.head()
coords_df.to_csv("team_coords.csv",index = False)